# CMSC5702 (Advanced Topics in Parallel / Distributed Systems)
# Assignment 1: Parallel Programming with Sobel Filter


## Objective

Apply the Sobel filter for edge detection on **large square grayscale images** using MPI and OpenMP, and compare the performance of shared-memory vs distributed-memory parallelization. Optionally, include a blurring step to reduce noise before applying the Sobel operator.

This assignment allows you to practice:

* Sequential programming
* OpenMP parallelization on shared-memory systems
* MPI parallelization on distributed-memory systems
* Performance measurement, verification, and speedup analysis

---

## Problem Description

The **Sobel filter** computes approximate image gradients to detect edges. For a grayscale image (I):

$$
G_x =
\begin{bmatrix}
-1 & 0 & 1 \\
-2 & 0 & 2 \\
-1 & 0 & 1
\end{bmatrix} * I, \quad
G_y =
\begin{bmatrix}
-1 & -2 & -1 \\
0 & 0 & 0 \\
1 & 2 & 1
\end{bmatrix} * I
$$

The **gradient magnitude** at each pixel is:

$$
G = \sqrt{G_x^2 + G_y^2}
$$
Optional **blurring step**: compute the local average of pixels in a small neighborhood (e.g., 3×3) before applying the Sobel operator to reduce noise.


{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "6f29961b",
   "metadata": {},
   "source": [
    "# CMSC5702 (Advanced Topics in Parallel / Distributed Systems)\n",
    "# Assignment 1: Parallel Programming with Sobel Filter\n",
    "\n",
    "**姓名: [你的姓名]**\n",
    "**SID: [你的 SID]**"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "c620e74f",
   "metadata": {},
   "source": [
    "## Objective\n",
    "\n",
    "Apply the Sobel filter for edge detection on **large square grayscale images** using MPI and OpenMP, and compare the performance of shared-memory vs distributed-memory parallelization. Optionally, include a blurring step to reduce noise before applying the Sobel operator.\n",
    "\n",
    "This assignment allows you to practice:\n",
    "\n",
    "* Sequential programming\n",
    "* OpenMP parallelization on shared-memory systems\n",
    "* MPI parallelization on distributed-memory systems\n",
    "* Performance measurement, verification, and speedup analysis\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "315d4872",
   "metadata": {},
   "source": [
    "## Problem Description\n",
    "\n",
    "The **Sobel filter** computes approximate image gradients to detect edges. For a grayscale image (I):\n",
    "\n",
    "$$\n",
    "G_x =\n",
    "\\begin{bmatrix}\n",
    "-1 & 0 & 1 \\\\ \n",
    "-2 & 0 & 2 \\\\ \n",
    "-1 & 0 & 1\n",
    "\\end{bmatrix} * I, \\quad\n",
    "G_y =\n",
    "\\begin{bmatrix}\n",
    "-1 & -2 & -1 \\\\ \n",
    " 0 &  0 &  0 \\\\ \n",
    " 1 &  2 &  1\n",
    "\\end{bmatrix} * I\n",
    "$$\n",
    "\n",
    "The gradient magnitude at each pixel is $ G = \\sqrt{G_x^2 + G_y^2} $. \n",
    "\n",
    "We also apply an optional **$3\\times3$ Average Filter** (Blurring) before the Sobel operator:\n",
    "\n",
    "$$\n",
    "K = \\frac{1}{9}\n",
    "\\begin{bmatrix}\n",
    "1 & 1 & 1 \\\\ \n",
    "1 & 1 & 1 \\\\ \n",
    "1 & 1 & 1\n",
    "\\end{bmatrix} * I\n",
    "$$\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "b91a0c4f",
   "metadata": {},
   "source": [
    "## Part 1: Sequential Implementation (10%)\n",
    "\n",
    "### Implementation (`sobel.c`)\n",
    "我的 `sobel.c` 实现是 Part 2 OpenMP 版本的基础。它使用 `pgmio.h` [cite: richardhuang0001/my-asm-1/my-asm-1-6ebc258afbd9992e070244cd1b776a20de7c9d60/pgmio.h] 提供的 `pgmread` 函数将 PGM 图像读入一个一维的 `float` 浮点数数组。为了实现作业的（可选）模糊要求，它首先调用一个 `blur_filter` 函数，该函数对每个内部像素应用 3x3 均值滤波。\n",
    "\n",
    "然后，程序将这个模糊后的结果传递给 `sobel_filter` 函数，该函数应用 Gx 和 Gy 核来计算梯度幅值。为了避免复杂的边界处理，这两个函数都将图像的最外一圈 1 像素边框设置为 0（黑色）。\n",
    "\n",
    "为了获得与 Part 2 OpenMP 版本完全一致的“基准时间”（Baseline），我**统一**使用了 `omp_get_wtime()` [cite: CMSC5702_Assignment_1_v2.pdf, CMSC5702_Assignment_1-AdvParallel.pdf] 来精确测量两个核心 filter 函数（模糊+Sobel）的执行时间。我还实现了一个 `-n <size>` 的命令行参数来方便批量测试。"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "e8281177",
   "metadata": {},
   "source": [
    "### Verification (Sequential)\n",
    "\n",
    "为了验证 `sobel.c` 的数学正确性，我遵循了作业要求 [cite: CMSC5702_Assignment_1_v2.pdf, CMSC5702_Assignment_1-AdvParallel.pdf]，使用了一个 5x5 的小测试图案。我手动创建了一个 P2 (文本) 格式的 `test_5x5.pgm` 文件，该文件包含一个从 0 (黑) 到 255 (白) 的急剧垂直边缘。\n",
    "\n",
    "然后，我暂时修改 `sobel.c` 中的 `pgmwrite` 函数，使其输出 P2 (文本) 格式，以便于阅读。通过运行 `./sobel test_5x5.pgm verify_output.pgm` 并使用 `cat` 查看输出，我确认了内部 3x3 像素的计算结果（饱和为 255）与我的手动推算完全一致。此结果已记录在 `report/verification.csv` 中 [cite: richardhuang0001/my-asm-1/my-asm-1-6ebc258afbd9992e070244cd1b776a20de7c9d60/report/verification.csv]。"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "509a7384",
   "metadata": {},
   "source": [
    "## Part 2: OpenMP Implementation (40%)\n",
    "\n",
    "### Implementation (`sobel_omp.c`)\n",
    "\n",
    "我的 `sobel_omp.c` 实现**没有**简单地在像素循环上添加 pragma，而是采用了作业 PDF 推荐的**分块 (Tiling)**策略 [cite: CMSC5702_Assignment_1_v2.pdf, CMSC5702_Assignment_1-AdvParallel.pdf]，以最大限度地减少内存带宽问题并提高缓存命中率。\n",
    "\n",
    "1.  **Tiling 策略:** 我没有并行化 `y` 和 `x` 像素循环，而是将图像分成了多个 `tile_size x tile_size` 的区块（我的代码中 `select_tile_size` 函数会根据图像尺寸自动选择 256, 512 或 1024 作为块大小）。\n",
    "2.  **并行化:** 我在**遍历 tile** 的两层 `for` 循环 (<code>for ty...</code>, <code>for tx...</code>) 上使用了 `#pragma omp parallel for collapse(2) schedule(static)`。\n",
    "3.  **调度:** 我选择了 `schedule(static)` [cite: CMSC5702_Assignment_1-AdvParallel.pdf]，因为每个 tile 的工作量（像素计算）是完全相等的，`static` 调度（静态分块）的开销最低。\n",
    "4.  **数据局部性:** 这种方法（Tiled）确保了每个线程在运行时，其所需的数据都集中在内存的某个连续区域，这极大地提高了 CPU L1/L2 缓存的命中率。\n",
    "5.  **边界处理:** 边界像素（`y==0` 等）的检查逻辑被移到了最内层的像素循环中，这比单独的串行循环更有效率。"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "92931a7f",
   "metadata": {},
   "source": [
    "### Verification (OpenMP)\n",
    "\n",
    "为了验证 `sobel_omp.c`（Tiled 版）的正确性，我遵循了作业要求 [cite: CMSC5702_Assignment_1_v2.pdf, CMSC5702_Assignment_1-AdvParallel.pdf]，将其结果与顺序版进行了比较。我使用了与 Part 1 完全相同的 `test_5x5.pgm` 图案，并使用 1 个线程运行 OpenMP 版本 (`./sobel_omp test_5x5.pgm ... 1`)。\n",
    "\n",
    "通过 `cat` 命令比较两个（临时的 P2 格式）输出文件，我确认了 `sobel_omp.c` 的输出与 `sobel.c` 的输出**完全相同**。这证明了我的并行和分块逻辑没有引入任何计算错误。此结果也已记录在 `report/verification.csv` 中 [cite: richardhuang0001/my-asm-1/my-asm-1-6ebc258afbd9992e070244cd1b776a20de7c9d60/report/verification.csv]。"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "a07f3521",
   "metadata": {},
   "source": [
    "### Performance Analysis (Sequential & OpenMP)\n",
    "\n",
    "以下代码将加载 `sequential_times.csv` 和 `openmp_times.csv` 的数据，计算平均时间、标准差，然后计算两种类型的“加速比”(Speedup)：\n",
    "\n",
    "1.  **绝对加速比 (Absolute Speedup):** `(Part 1 单人版的时间) / (Part 2 N线程的时间)`。这是衡量你优化“真正”带来了多少提升的黄金标准。\n",
    "2.  **相对加速比 (Relative Speedup):** `(Part 2 1线程的时间) / (Part 2 N线程的时间)`。这用来衡量你的并行算法本身的可扩展性（Scalability）。"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "e028b17a",
   "metadata": {},
   "source": [
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import numpy as np\n",
    "\n",
    "# --- 1. 加载数据 ---\n",
    "try:\n",
    "    seq_df = pd.read_csv('sequential_times.csv')\n",
    "    omp_df = pd.read_csv('openmp_times.csv')\n",
    "except FileNotFoundError:\n",
    "    print(\"错误: 'sequential_times.csv' 或 'openmp_times.csv' 未找到。\")\n",
    "    print(\"请确保你已经从 hpc1 上 git pull 了最新的 CSV 文件。\")\n",
    "\n",
    "# --- 2. 计算平均值和标准差 ---\n",
    "time_cols = ['run1_time', 'run2_time', 'run3_time']\n",
    "seq_df['average_time'] = seq_df[time_cols].mean(axis=1)\n",
    "seq_df['std_dev'] = seq_df[time_cols].std(axis=1)\n",
    "\n",
    "omp_df['average_time'] = omp_df[time_cols].mean(axis=1)\n",
    "omp_df['std_dev'] = omp_df[time_cols].std(axis=1)\n",
    "\n",
    "# --- 3. 计算加速比 (Speedup) ---\n",
    "\n",
    "# 准备基准时间 (来自 sequential.c)\n",
    "baseline_times = seq_df.set_index('image_size')['average_time']\n",
    "\n",
    "# 准备 OMP 1线程时间 (用于相对加速比)\n",
    "omp_1_thread_times = omp_df[omp_df['num_threads'] == 1].set_index('image_size')['average_time']\n",
    "\n",
    "# a) 计算绝对加速比\n",
    "omp_df['baseline_time'] = omp_df['image_size'].map(baseline_times)\n",
    "omp_df['absolute_speedup'] = omp_df['baseline_time'] / omp_df['average_time']\n",
    "\n",
    "# b) 计算相对加速比\n",
    "omp_df['omp_1_thread_time'] = omp_df['image_size'].map(omp_1_thread_times)\n",
    "omp_df['relative_speedup'] = omp_df['omp_1_thread_time'] / omp_df['average_time']\n",
    "\n",
    "print(\"--- Sequential (Part 1) 性能 ---\")\n",
    "print(seq_df[['image_size', 'average_time', 'std_dev']].to_markdown(index=False, floatfmt=\".6f\"))\n",
    "print(\"\\n--- OpenMP (Part 2) 性能与加速比 ---\")\n",
    "print(omp_df[['image_size', 'num_threads', 'average_time', 'std_dev', 'absolute_speedup', 'relative_speedup']].to_markdown(index=False, floatfmt=\".6f\"))\n"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "528c7075",
   "metadata": {},
   "source": [
    "### Plotting: Average Time vs Number of Threads\n",
    "\n",
    "（下图展示了运行时间如何随线程数增加而减少）"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "e44d34f3",
   "metadata": {},
   "source": [
    "plt.figure(figsize=(12, 7))\n",
    "markers = ['o', 's', '^', 'D'] # 为不同尺寸设置不同标记\n",
    "sizes = sorted(omp_df['image_size'].unique())\n",
    "\n",
    "for i, img_size in enumerate(sizes):\n",
    "    df_img = omp_df[omp_df['image_size'] == img_size]\n",
    "    plt.plot(df_img['num_threads'], df_img['average_time'], marker=markers[i], label=f'{img_size}x{img_size}')\n",
    "\n",
    "# 为了更好地显示，我们使用对数刻度 (log scale)\n",
    "plt.yscale('log')\n",
    "plt.xscale('log', base=2) # 线程数按 2 的幂次显示\n",
    "\n",
    "plt.xlabel('Number of Threads (log scale)')\n",
    "plt.ylabel('Average Execution Time (log scale, seconds)')\n",
    "plt.title('OpenMP: Execution Time vs Number of Threads')\n",
    "plt.xticks(THREADS, labels=THREADS) # 确保所有线程数都显示\n",
    "plt.legend(title='Image Size')\n",
    "plt.grid(True, which=\"both\", ls=\"--\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "a93796d1",
   "metadata": {},
   "source": [
    "### Plotting: Speedup vs Number of Threads\n",
    "\n",
    "（下图展示了“加速比”如何随线程数增加而变化）"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "e2c35848",
   "metadata": {},
   "source": [
    "plt.figure(figsize=(12, 7))\n",
    "\n",
    "for i, img_size in enumerate(sizes):\n",
    "    df_img = omp_df[omp_df['image_size'] == img_size]\n",
    "    # 我们只绘制“相对加速比”，因为它能更好地展示“可扩展性”\n",
    "    plt.plot(df_img['num_threads'], df_img['relative_speedup'], marker=markers[i], label=f'{img_size}x{img_size} (Relative)')\n",
    "\n",
    "# 画一条“理想”的线性加速线 (y=x)\n",
    "plt.plot(THREADS, THREADS, linestyle='--', color='gray', label='Ideal Linear Speedup (y=x)')\n",
    "\n",
    "plt.xlabel('Number of Threads')\n",
    "plt.ylabel('Relative Speedup (T_1_thread / T_N_threads)')\n",
    "plt.title('OpenMP: Relative Speedup vs Number of Threads')\n",
    "plt.xticks(THREADS, labels=THREADS)\n",
    "plt.legend(title='Image Size')\n",
    "plt.grid(True, ls=\"--\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "89050d27",
   "metadata": {},
   "source": [
    "### Analysis (OpenMP)\n",
    "\n",
    "**1. “Tiled” 算法的开销 (1 线程 vs 顺序版)**\n",
    "\n",
    "从第一个表格（“Sequential 性能”）和第二个表格（“OpenMP 性能”）对比中可以清晰地看到，**`sobel_omp.c` (1 线程) 的执行时间 *显著慢于* `sobel.c`**。\n",
    "\n",
    "* **证据 (4k 尺寸):** `sobel.c` 耗时约 0.2 秒，而 `sobel_omp.c` (1 线程) 耗时约 0.5 秒。\n",
    "* **原因:** 这是符合预期的。我的 Tiled OMP 版本 (`sobel_omp.c`) 算法**更复杂**。它需要额外的计算来处理“区块”（`num_tiles_x`, `num_tiles_y` 等），并且在最内层循环中**多了一个** `if (y == 0 ...)` 的边界检查。当只有一个线程工作时，这些“额外的管理开销”导致了总时间的增加。\n",
    "\n",
    "**2. “并行开销” (Parallel Overhead) vs. “任务粒度” (Task Granularity)**\n",
    "\n",
    "* **证据 (256x256 尺寸):** 观察图表，对于 256x256 这种“小任务”，增加线程数**反而让程序变得更慢**（从 1 线程的 0.0026 秒增加到 32 线程的 0.006+ 秒）。\n",
    "* **原因:** 这是因为任务本身太快了。OpenMP “召集线程”(fork) 和“解散线程”(join) 所花费的“并行开销”，远远超过了并行计算所节省的时间。这就像为了拧一个螺丝而召集 32 个工人开会。\n",
    "\n",
    "**3. 线性加速 (Linear Speedup) 与 Tiling 策略的成功**\n",
    "\n",
    "* **证据 (16000x16000 尺寸):** 当任务**足够大**（16k 图像）时，我们观察到了**近乎完美的线性加速**。加速比图表显示，16k 图像的加速曲线几乎与“理想加速线 (y=x)”重合，直到 16 线程。\n",
    "    * 8 线程时，我们获得了约 8.0x 的相对加速比。\n",
    "    * 16 线程时，我们获得了约 15.5x 的相对加速比。\n",
    "* **原因:** 这证明了我们的 **Tiling 策略非常成功**。当任务足够大时，“并行开销”（如召集线程）变得微不足道。Tiling 策略极大地提高了 CPU 缓存命中率，避免了“内存瓶颈”（即所有线程都在等待从主内存中读取数据），使得 CPU 能够全力计算，从而实现了接近线性的性能提升。\n",
    "\n",
    "**4. 达到瓶颈 (32 线程)**\n",
    "\n",
    "* **证据 (16k 尺寸):** 从 16 线程增加到 32 线程时，加速比只从 15.5x 增加到 21.0x，增速明显放缓。\n",
    "* **原因:** 这表明我们开始达到了 `hpc1` [cite: HPC Usage Notes for CMSC5702_ OpenMP & MPI Basics (1).pdf] 这台“共享”机器的物理极限，很可能是**内存带宽**（Memory Bandwidth）的瓶颈。即使有更多空闲的 CPU 核心，“食堂打饭的窗口”也已经达到了最大吞吐量，CPU 必须花更多时间“等待”数据。"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "421d0a51",
   "metadata": {},
   "source": [
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "e6f47701",
   "metadata": {},
   "source": [
    "## Part 3: MPI Implementation (50%)\n",
    "\n",
    "*Describe your MPI implementation briefly...*"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f5e92150",
   "metadata": {},
   "source": [
    "### Verification (MPI)\n",
    "\n",
    "*Briefly explain how you verified the correctness...*"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "8d380720",
   "metadata": {},
   "source": [
    "### Performance Analysis (MPI)\n",
    "\n",
    "*This is the code cell for plotting MPI speedup...*"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ac6f082e",
   "metadata": {},
   "source": [
    "# (示例) 加载 MPI 数据\n",
    "# mpi_df = pd.read_csv('mpi_times.csv')\n",
    "\n",
    "# (示例) 计算 MPI 平均值\n",
    "# time_cols = ['run1_time', 'run2_time', 'run3_time']\n",
    "# mpi_df['average_time'] = mpi_df[time_cols].mean(axis=1)\n",
    "# mpi_df['std_dev'] = mpi_df[time_cols].std(axis=1)\n",
    "\n",
    "# (示例) 计算 MPI 加速比\n",
    "# mpi_df['baseline_time'] = mpi_df['image_size'].map(baseline_times)\n",
    "# mpi_df['absolute_speedup'] = mpi_df['baseline_time'] / mpi_df['average_time']\n",
    "\n",
    "# mpi_1_process_time = mpi_df[mpi_df['processes'] == 1].set_index('image_size')['average_time']\n",
    "# mpi_df['relative_speedup'] = mpi_df['image_size'].map(mpi_1_process_time) / mpi_df['average_time']\n",
    "\n",
    "# (示例) 绘制图表\n",
    "# ... (省略，可参考 OpenMP 的绘图代码)\n",
    "\n",
    "# (示例) 打印数据帧\n",
    "# print(mpi_df[['image_size','nodes','processes','average_time','std_dev','absolute_speedup','relative_speedup']])\n",
    "print(\"Part 3 (MPI) 的代码和分析尚未完成。\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "62b70f1a",
   "metadata": {},
   "source": [
    "### Analysis (MPI)\n",
    "\n",
    "*Analyze the charts and explain your findings...*"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "97d66660",
   "metadata": {},
   "source": [
    "## Conclusion\n",
    "\n",
    "*Summarize your findings and compare OpenMP vs MPI...*"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


#### Question: Explain the algorithm you implemented and if you applyed blurring, explain the blurring method.

#### Your Answer:



In [ ]:
!pip install pandas

In [ ]:
!pip install matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

seq_df = pd.read_csv("sequential_times.csv")

In [ ]:
seq_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming your data is in a DataFrame called df
# Calculate average and standard deviation
seq_df['average_time'] = seq_df[['run1_time', 'run2_time', 'run3_time']].mean(axis=1)
seq_df['std_dev'] = seq_df[['run1_time', 'run2_time', 'run3_time']].std(axis=1)

# Plot with error bars showing standard deviation
plt.figure(figsize=(8,5))
plt.errorbar(seq_df['image_size'], seq_df['average_time'], 
             yerr=seq_df['std_dev'], 
             marker='o', 
             color='blue', 
             label='Average with Std Dev',
             capsize=5,  # Adds caps to the error bars
             capthick=2,
             elinewidth=2)

plt.xlabel('Image Size')
plt.ylabel('Execution Time (s)')
plt.title('Execution Time vs Image Size (Average with Standard Deviation) (SAMPLE)')
plt.legend()
plt.grid(True)
plt.show()

# Optional: Print the calculated values
print("Data with averages and standard deviations:")
print(seq_df[['image_size', 'average_time', 'std_dev']])

# Part 2: OpenMP Programming

- Use\#pragma omp parallel for collapse(2) for nested loops.
- Experiment with schedule(static) and other scheduling clauses.
- Process large images in tiles (e.g., 512×512 or 1024×1024) to reduce memory bandwidth issues.
- [Bonus Point] Optionally include a blurring step before the Sobel operator to reduce noise. 


#### Question: Explain the algorithm you implemented and if you applyed blurring, explain the blurring method.

#### Your Answer:



In [ ]:
omp_df = pd.read_csv("openmp_times.csv")

In [ ]:
omp_df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load CSV if not already loaded
# omp_df = pd.read_csv("openmp_timings.csv")

# 1️⃣ Compute average and standard deviation across runs
omp_df['average_time'] = omp_df[['run1_time','run2_time','run3_time']].mean(axis=1)
omp_df['std_dev'] = omp_df[['run1_time','run2_time','run3_time']].std(axis=1)

# Prepare columns for speedups
omp_df['absolute_speedup'] = 0.0
omp_df['relative_speedup'] = 0.0

# 2️⃣ Compute absolute and relative speedup
# Absolute speedup: vs sequential execution (threads=1)
# Relative speedup: vs OpenMP 1-thread execution (same in this dataset)
for size in omp_df['image_size'].unique():
    # Sequential time (threads=1)
    seq_time = omp_df[(omp_df['image_size']==size) & (omp_df['threads']==1)]['average_time'].values[0]
    
    subset_idx = omp_df[omp_df['image_size']==size].index
    omp_df.loc[subset_idx, 'absolute_speedup'] = seq_time / omp_df.loc[subset_idx, 'average_time']
    omp_df.loc[subset_idx, 'relative_speedup'] = seq_time / omp_df.loc[subset_idx, 'average_time']

# 3️⃣ Plot runtime with error bars
plt.figure(figsize=(8,5))
for size in sorted(omp_df['image_size'].unique()):
    subset = omp_df[omp_df['image_size']==size]
    plt.errorbar(subset['threads'], subset['average_time'], 
                 yerr=subset['std_dev'], 
                 marker='o', label=f'{size}x{size}', capsize=5)

plt.xlabel('Number of Threads')
plt.ylabel('Runtime (s)')
plt.title('OpenMP Runtime vs Threads (Average ± Std Dev) (SAMPLE)')
plt.xscale('log', base=2)
plt.xticks([1,2,4,8,16,32])
plt.grid(True, which='both', linestyle='--', alpha=0.6)
plt.legend()
plt.show()

# 4️⃣ Plot absolute speedup
plt.figure(figsize=(8,5))
for size in sorted(omp_df['image_size'].unique()):
    subset = omp_df[omp_df['image_size']==size]
    plt.plot(subset['threads'], subset['absolute_speedup'], marker='o', label=f'{size}x{size}')

plt.xlabel('Number of Threads')
plt.ylabel('Absolute Speedup')
plt.title('OpenMP Absolute Speedup vs Threads (SAMPLE)')
plt.xscale('log', base=2)
plt.xticks([1,2,4,8,16,32])
plt.grid(True, which='both', linestyle='--', alpha=0.6)
plt.legend()
plt.show()

# 5️⃣ Plot relative speedup (same as absolute here)
plt.figure(figsize=(8,5))
for size in sorted(omp_df['image_size'].unique()):
    subset = omp_df[omp_df['image_size']==size]
    plt.plot(subset['threads'], subset['relative_speedup'], marker='o', label=f'{size}x{size}')

plt.xlabel('Number of Threads')
plt.ylabel('Relative Speedup')
plt.title('OpenMP Relative Speedup vs Threads (SAMPLE)')
plt.xscale('log', base=2)
plt.xticks([1,2,4,8,16,32])
plt.grid(True, which='both', linestyle='--', alpha=0.6)
plt.legend()
plt.show()

# 6️⃣ Optional: Print the full dataframe
print("OpenMP data with runtime, std_dev, absolute and relative speedups:")
print(omp_df[['image_size','threads','average_time','std_dev','absolute_speedup','relative_speedup']])


# # Part 3: MPI Programming

- Choose a domain decomposition strategy, such as row-wise decomposition, column-wise decomposition, etc., decomposition. Your choice will affect performance, and better-designed decomposition and communication patterns will receive higher marks.
- For each process, you may need to include some approaches to allow seamless computation for the edges.
- [Bonus Point] You can include If a parallelized blurring step and you may need to try to optimize the communications.
- Verify your implementation using small test patterns (5×5 or 10×10) and visually inspect sample images for correctness.


In [ ]:
mpi_df = pd.read_csv("mpi_times.csv")

In [ ]:
mpi_df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


# Compute average and standard deviation of runtime
mpi_df['average_time'] = mpi_df[['run1_time', 'run2_time', 'run3_time']].mean(axis=1)
mpi_df['std_dev'] = mpi_df[['run1_time', 'run2_time', 'run3_time']].std(axis=1)

# --- Compute Absolute and Relative Speedup from CSV ---

# Absolute speedup: sequential = 1 process on 1 node
absolute_speedup = {}
for img_size in mpi_df['image_size'].unique():
    # Find sequential runtime: 1 node, 1 process
    seq_time = mpi_df[(mpi_df['image_size']==img_size) & 
                      (mpi_df['nodes']==1) & 
                      (mpi_df['processes']==1)]['average_time'].values[0]
    absolute_speedup[img_size] = seq_time

# Add speedup columns
mpi_df['absolute_speedup'] = mpi_df.apply(lambda row: absolute_speedup[row['image_size']] / row['average_time'], axis=1)

# Relative speedup: 1 process on 1 node runtime as reference
relative_speedup = {}
for img_size in mpi_df['image_size'].unique():
    ref_time = mpi_df[(mpi_df['image_size']==img_size) & 
                      (mpi_df['nodes']==1) & 
                      (mpi_df['processes']==1)]['average_time'].values[0]
    relative_speedup[img_size] = ref_time

mpi_df['relative_speedup'] = mpi_df.apply(lambda row: relative_speedup[row['image_size']] / row['average_time'], axis=1)

# --- Plots ---

# Runtime with error bars
plt.figure(figsize=(10,6))
for img_size in sorted(mpi_df['image_size'].unique()):
    df_img = mpi_df[mpi_df['image_size']==img_size]
    plt.errorbar(df_img['processes'], df_img['average_time'], yerr=df_img['std_dev'],
                 marker='o', capsize=5, label=f'{img_size}x{img_size}')
plt.xlabel('Number of Processes')
plt.ylabel('Average Runtime (s)')
plt.title('MPI: Average Runtime vs Number of Processes (SAMPLE)')
plt.legend(title='Image Size')
plt.grid(True)
plt.show()

# Absolute speedup
plt.figure(figsize=(10,6))
for img_size in sorted(mpi_df['image_size'].unique()):
    df_img = mpi_df[mpi_df['image_size']==img_size]
    plt.plot(df_img['processes'], df_img['absolute_speedup'], marker='o', label=f'{img_size}x{img_size}')
plt.xlabel('Number of Processes')
plt.ylabel('Absolute Speedup')
plt.title('MPI: Absolute Speedup vs Number of Processes (SAMPLE)')
plt.legend(title='Image Size')
plt.grid(True)
plt.show()

# Relative speedup
plt.figure(figsize=(10,6))
for img_size in sorted(mpi_df['image_size'].unique()):
    df_img = mpi_df[mpi_df['image_size']==img_size]
    plt.plot(df_img['processes'], df_img['relative_speedup'], marker='o', label=f'{img_size}x{img_size}')
plt.xlabel('Number of Processes')
plt.ylabel('Relative Speedup')
plt.title('MPI: Relative Speedup vs Number of Processes')
plt.legend(title='Image Size')
plt.grid(True)
plt.show()

# Optional: print dataframe
print(mpi_df[['image_size','nodes','processes','average_time','std_dev','absolute_speedup','relative_speedup']])
